### Get a list of OpenSea collections.

In [15]:
### 这里的代码是取一次，最多取100个
import requests

# API endpoint
url = "https://api.opensea.io/api/v2/collections"

# Headers with API key
headers = {
    "accept": "application/json",
    "x-api-key": "af06c28ee5414d31b2be1ab513d826af"  # Replace with your actual API key
}

# Optional query parameters
params = {
    "chain": "ethereum",          # Filter by blockchain (e.g., Ethereum)
    "limit": 100,                 # Limit to 10 collections
    "include_hidden": False,     # Exclude hidden collections
    # "creator_username": "username",  # Uncomment to filter by creator
    # "next": "cursor_value",         # Uncomment for pagination
    # "order_by": "created_date"      # Uncomment to sort
}

try:
    # Make the GET request
    response = requests.get(url, headers=headers, params=params,verify=False)
    
    # Check if the request was successful
    response.raise_for_status()
    
    # Parse and print the JSON response
    data = response.json()
    collections = data.get("collections", [])
    

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except requests.exceptions.RequestException as err:
    print(f"Error occurred: {err}")

In [16]:
collections

[{'collection': 'rtertert',
  'name': 'rtertert',
  'description': '',
  'image_url': 'https://i.seadn.io/s/raw/files/36fd7707fc8bb1f4cb231aadbd693403.jpg?w=500&auto=format',
  'banner_image_url': '',
  'owner': '0x65ea660733580ea259507bf25824eaa5a3ab19ac',
  'safelist_status': 'not_requested',
  'category': '',
  'is_disabled': False,
  'is_nsfw': False,
  'trait_offers_enabled': False,
  'collection_offers_enabled': True,
  'opensea_url': 'https://opensea.io/collection/rtertert',
  'project_url': '',
  'wiki_url': '',
  'discord_url': '',
  'telegram_url': '',
  'twitter_username': '',
  'instagram_username': '',
  'contracts': [{'address': '0x8fa20e2de87ecf640545ec98b7f602eee1cc1bdb',
    'chain': 'ethereum'}]},
 {'collection': 'erterer-1',
  'name': 'erterer',
  'description': '',
  'image_url': 'https://i.seadn.io/s/raw/files/b181942136a0ad91a339be55eb33d07b.jpg?w=500&auto=format',
  'banner_image_url': '',
  'owner': '0x575a3b4233b26b52e4afbb6480f6cd4431e4fd16',
  'safelist_statu

### 下面这个代码是取一个链上所有collection信息的，爬以太坊的爬了十万条也没爬完

In [20]:

import requests
import time
import pandas as pd
import json
requests.packages.urllib3.disable_warnings()  # 关闭 SSL 警告信息

# OpenSea API 端点
url = "https://api.opensea.io/api/v2/collections"

# 请求头信息，包含 API Key（请替换为你自己的 API Key）
headers = {
    "accept": "application/json",
    "x-api-key": "af06c28ee5414d31b2be1ab513d826af"  # 替换为你的 API 密钥
}

# 请求参数
params = {
    "chain": "ethereum",       # 指定区块链类型为以太坊
    "limit": 100,              # 每页最多返回 100 条数据
    "include_hidden": False    # 是否包含被隐藏的 collection，False 表示不包含
}

# 初始化变量
all_collections = []  # 用于保存所有获取到的集合
next_cursor = None    # 用于分页的游标（如果有下一页，API 会返回此字段）

# 开始循环抓取所有分页数据
while True:
    if next_cursor:
        # 如果有下一页游标，加入到请求参数中
        params["next"] = next_cursor

    try:
        # 发送 GET 请求
        response = requests.get(url, headers=headers, params=params, verify=False)

        if response.status_code == 200:
            # 请求成功，解析返回数据
            data = response.json()
            collections = data.get("collections", [])  # 当前页的集合
            all_collections.extend(collections)        # 添加到总列表
            next_cursor = data.get("next")             # 获取下一页游标（如果存在）

            print(f"Got {len(collections)} collections, total: {len(all_collections)}")

            if not next_cursor:
                # 如果没有下一页，终止循环
                break

        elif response.status_code == 429:
            # 如果遇到速率限制，等待 10 秒后继续
            print("Rate limit hit, waiting 10 seconds...")
            time.sleep(10)
            continue
        else:
            # 其他错误，打印状态码和错误信息
            print(f"Error: {response.status_code}, {response.text}")
            break

        # 每次请求之间稍作延时，避免触发速率限制
        time.sleep(1)

    except KeyboardInterrupt:
        # 如果用户中断程序（Ctrl+C），保存已获取的数据到本地文件
        print("Interrupted, saving data...")
        with open("collections.json", "w") as f:
            json.dump(all_collections, f)
        print("Saved to collections.json")
        break

    except:
        # 其他异常，保存已获取的数据以避免丢失
        print("Error occurred, saving data...")
        with open("collections.json", "w") as f:
            json.dump(all_collections, f)
        print("Saved to collections.json")
        break

Got 100 collections, total: 100
Got 100 collections, total: 200
Got 100 collections, total: 300
Got 100 collections, total: 400
Interrupted, saving data...
Saved to collections.json


In [ ]:
all_collections_df = pd.json_normalize(all_collections, sep='_')

In [22]:
all_collections_df

,collection,name,description,image_url,banner_image_url,owner,safelist_status,category,is_disabled,is_nsfw,trait_offers_enabled,collection_offers_enabled,opensea_url,project_url,wiki_url,discord_url,telegram_url,twitter_username,instagram_username,contracts
0,ertertert-1,ertertert,,https://i.seadn.io/s/raw/files/61761dc88bdb90b...,,0x1a7ea8d5678ba65d26e36a5792012f10f16ab87b,not_requested,,False,False,False,True,https://opensea.io/collection/ertertert-1,,,,,,,[{'address': '0x84c97699ed7dda5f020fd615e5feac...
1,sb15-1,SB15,... ... ....\n\nMade with [NFTs2Me.com](https:...,https://i.seadn.io/s/raw/files/b0e3a7df5dde8c5...,,0xd9b891fb49091103a4a113c325b34dd42dd3c2f2,not_requested,,False,False,False,True,https://opensea.io/collection/sb15-1,https://0x2174908334d31155dA817D64395fDa1AF375...,,,,,,[{'address': '0x2174908334d31155da817d64395fda...
2,singe-heureux,singe heureux,,https://i.seadn.io/s/raw/files/836d0fe1e384a6b...,,0xd5eafee59726d9c79105183cd55ece2d97e49737,not_requested,,False,False,False,False,https://opensea.io/collection/singe-heureux,,,,,,,[{'address': '0x532f6639d154409cb49088941118f2...
3,rtertert,rtertert,,https://i.seadn.io/s/raw/files/36fd7707fc8bb1f...,,0x65ea660733580ea259507bf25824eaa5a3ab19ac,not_requested,,False,False,False,True,https://opensea.io/collection/rtertert,,,,,,,[{'address': '0x8fa20e2de87ecf640545ec98b7f602...
4,erterer-1,erterer,,https://i.seadn.io/s/raw/files/b181942136a0ad9...,,0x575a3b4233b26b52e4afbb6480f6cd4431e4fd16,not_requested,,False,False,False,True,https://opensea.io/collection/erterer-1,,,,,,,[{'address': '0xf029639ddd94e065df1107ae308874...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,anofk,Anofk,,,,0x94da09fff46ee54697e4aab8df09b113e6553f7e,not_requested,,False,False,False,True,https://opensea.io/collection/anofk,,,,,,,[{'address': '0xcbb7275086411df55501364164ac28...
396,holoyak-10,HoloYak,,https://i.seadn.io/s/raw/files/4482666435d4417...,,0xb075c39a2adc2727aa631c605c05cd73b83ede02,not_requested,,False,False,False,True,https://opensea.io/collection/holoyak-10,,,,,,,[{'address': '0xb969ad3dff2e8cd3d1e6cb7ffd3a12...
397,ethernova-11,EtherNova,,https://i.seadn.io/s/raw/files/26d09127e1ba2d1...,,0x1c8645942a53025415d83ba78f3c84f1138a16d4,not_requested,,False,False,False,True,https://opensea.io/collection/ethernova-11,,,,,,,[{'address': '0xc951687eaae15a12e3157cc79e4d99...
398,gerberian-shepsky-2,Gerberian Shepsky,,,,0xf3a47101876c6797960577d4d3bafea08df2d74b,not_requested,,False,False,False,False,https://opensea.io/collection/gerberian-shepsky-2,,,,,,,[{'address': '0x0665845ed102170e63251c671eedd8...


### Get Events(by collection) 爬取某个collection在一段时间内的交易信息

In [4]:
import requests
import time
import pandas as pd
from datetime import datetime, timezone
requests.packages.urllib3.disable_warnings() # disable warnings

# 替换成你的 API Key（如果有）
API_KEY = "af06c28ee5414d31b2be1ab513d826af"

# 这里换成你要查的 Collection  
# 可以在某个 NFT 的 URL 中找到 slug，例如 https://opensea.io/collection/doodles-official，slug 就是 "doodles-official"
COLLECTION_SLUG = "doodles-official"  


# 假设你的时间是 2025-03-01 00:00:00 UTC
dt1 = datetime(2025, 3, 1, 0, 0, 0, tzinfo=timezone.utc)
dt2 = datetime(2025, 4, 1, 0, 0, 0, tzinfo=timezone.utc)
# 转为 Unix 时间戳
AFTER_TIMESTAMP = int(dt1.timestamp())
BEFORE_TIMESTAMP = int(dt2.timestamp())

# API URL
url = f"https://api.opensea.io/api/v2/events/collection/{COLLECTION_SLUG}"

# 请求头
# 可以加agent，避免被封
headers = {
    'User-agent': 'Mozilla/5.0 (compatible; U; ABrowse 0.6; Syllable) AppleWebKit/420+ (KHTML, like Gecko)',
    "accept": "application/json",
    "x-api-key": API_KEY  # 如果你有 API Key，添加这个
}

# 请求参数
params = {
    "after": AFTER_TIMESTAMP,  # 起始时间（Unix 时间戳）
    "before": BEFORE_TIMESTAMP,  # 结束时间
    "event_type": ["sale"],  # 只获取交易（sale）事件
    "limit": 50  # 每次最多返回 50 条
}

# 结果存储
all_events = []
next_cursor = None

while True:
    if next_cursor:
        params["next"] = next_cursor  # 翻页

    response = requests.get(url, headers=headers, params=params,verify=False)
    
    if response.status_code == 200:
        data = response.json()
        events = data.get("asset_events", [])
        all_events.extend(events)

        # 分页：检查是否还有下一页
        next_cursor = data.get("next")
        if not next_cursor:
            break  # 没有更多数据了，退出循环
    else:
        print("请求失败:", response.status_code, response.text)
        break  # 发生错误时退出

    time.sleep(1)  # 避免触发 API 速率限制


In [ ]:
import json

# 保存为 JSON 文件
with open('doodles-official_nft_events.json', 'w', encoding='utf-8') as f:
    json.dump(all_events, f, ensure_ascii=False, indent=4)
print("数据已保存到 doodles-official_nft_events.json")

In [5]:
### 将 JSON 数据转换为 DataFrame
all_events_df = pd.json_normalize(all_events, sep='_')

In [25]:
all_events_df

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft_display_animation_url,nft_metadata_url,nft_opensea_url,nft_updated_at,nft_is_disabled,nft_is_nsfw,payment_quantity,payment_token_address,payment_decimals,payment_symbol
0,sale,0xcb96928a158a8692fd30c6066f097c904ca7de7b52dc...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1743461579,1,0x2d198f5f809691a66bc487ca572b22937b231b07,0x05a64fc3398b49b1dcbb45109ff6debf2ea3dc24,0xdd05070239444b8d678e1a40067678a572768ba286de...,1743461579,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-03-31T22:53:05.397952,False,False,3065000000000000000,0x0000000000000000000000000000000000000000,18,ETH
1,sale,,ethereum,,1743454463,1,0x3772fad0e4de82f27620c946e7f52946332ea02c,0x85d2c733e11e095b0adaa3c055b31fdd4168af2c,0xc1b2cc2cbfa62b66276a1084f257eb49c7100f35a8f1...,1743454463,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-04-07T02:52:03.601317,False,False,2960000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
2,sale,0xc9b32c5cda29594e2e8e4bdab548913ddb2e68e44910...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1743454451,1,0xa82801ddf9f5b974085e0450bebdd95365f5c3ab,0x3772fad0e4de82f27620c946e7f52946332ea02c,0x9879b2761b49f801f8d7346d45350ab3cb1b62e98bab...,1743454451,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-04-07T02:52:03.601317,False,False,2940000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
3,sale,,ethereum,,1743454343,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xa82801ddf9f5b974085e0450bebdd95365f5c3ab,0x6841edf41d82618278d912b54851664388cd8f72f072...,1743454343,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-04-07T02:52:03.601317,False,False,2960000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
4,sale,,ethereum,,1743454331,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x1ea27bce786a81022dfc156059771e8d3279a9a6,0x9f3a3cab23612a8ea31e12dbc452da9ab8a6c6101431...,1743454331,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-04-08T16:42:37.954892,False,False,2970000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,sale,0x2e5cefee957f16fa19cca934ff30633a59a1f591ccbd...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1740838139,1,0xdddd696973fd58db7aec22f96fbdcf40371e9fb1,0x4a1721c493c481b48ab05db82265e149c95ac33f,0xbcbb4729168a2245b7a191cf84f1b33bb1fe8a584486...,1740838139,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-03-04T16:34:49.723179,False,False,3200000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
805,sale,,ethereum,,1740835607,1,0x0d53cb5115db5965b66a43f09692750de174fe91,0x6eced2947c674c388b638564848aadf2f05ea974,0x26f536f33d5df202e01159dedbe527384b170fa96118...,1740835607,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-03-16T10:58:27.026730,False,False,3287900000000000000,0x0000000000000000000000000000000000000000,18,ETH
806,sale,0x7f8e16bcc01cbab03e7a35be421b5f2e7313dcf24960...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1740823991,1,0xf9bce92da0d4b8253077a6851a32bbd059e2a5a4,0x29e3c6eda0caf630b76493e3e6ea0adf9257f10c,0xf7612dc23a4bc7d6045ff7d4b7ec56a3d6d665d83a3b...,1740823991,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-03-01T10:13:13.862119,False,False,3398000000000000000,0x0000000000000000000000000000000000000000,18,ETH
807,sale,,ethereum,,1740821303,1,0x5b468edb7688e9ae6c1fa5a6d2debbef06e92907,0xf9091

### get nfts by collection 在上面爬取完某个collection的交易信息基础上爬取某个系列nft的具体信息

In [6]:
from tqdm import tqdm
import requests
import pandas as pd
import time

# -------------------------
# 请求配置
# -------------------------
headers = {
    "accept": "application/json",
    "x-api-key": "af06c28ee5414d31b2be1ab513d826af"
}
session = requests.Session()

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

# -------------------------
# 获取单个 NFT 元数据
# -------------------------
def get_nft(chain, contract, identifier):
    url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{contract}/nfts/{identifier}"
    try:
        resp = session.get(url, headers=headers, timeout=10)
        if resp.status_code == 200:
            return resp.json().get("nft")
        elif resp.status_code == 429:
            print("Rate limit hit. Sleeping 10s.")
            time.sleep(10)
            return None
        else:
            print(f"Failed {resp.status_code}: {resp.text}")
            return None
    except Exception as e:
        print(f"Request error for {identifier}: {e}")
        return None

# -------------------------
# 去重并准备数据
# -------------------------
unique_nfts_df = all_events_df[["nft_identifier", "chain", "nft_contract"]].drop_duplicates(subset="nft_identifier")
print(f"🔍 Total unique NFTs to fetch: {len(unique_nfts_df)}")

nfts = []
fails = []

# 可选：记录已爬取的 ID，防止二次抓时浪费
fetched_identifiers = set()

# -------------------------
# 开始爬取
# -------------------------
for i in tqdm(unique_nfts_df.index, desc="Fetching unique NFT details"):
    row = unique_nfts_df.loc[i]
    chain = row["chain"]
    contract = row["nft_contract"]
    identifier = str(row["nft_identifier"])

    if identifier in fetched_identifiers:
        continue

    if not all([chain, contract, identifier]):
        fails.append({"chain": chain, "contract": contract, "identifier": identifier})
        continue

    nft = get_nft(chain, contract, identifier)
    if nft:
        nft["nft_identifier"] = identifier
        nft["chain"] = chain
        nft["nft_contract"] = contract
        nfts.append(nft)
        fetched_identifiers.add(identifier)
    else:
        fails.append({"chain": chain, "contract": contract, "identifier": identifier})

    time.sleep(0.3)  # 限流

# -------------------------
# 保存数据
# -------------------------
nfts_df = pd.json_normalize(nfts, sep="_")
fails_df = pd.DataFrame(fails)



session.close()



🔍 Total unique NFTs to fetch: 347


Fetching unique NFT details:   0%|          | 0/347 [00:00<?, ?it/s]

Fetching unique NFT details: 100%|██████████| 347/347 [06:06<00:00,  1.06s/it]


In [ ]:
nfts_df.to_csv("unique_nfts_details.csv", index=False)
fails_df.to_csv("failed_nfts.csv", index=False)

In [19]:
nfts_df

,identifier,collection,contract,token_standard,name,description,image_url,display_image_url,display_animation_url,metadata_url,...,chain,nft_contract,rarity_strategy_id,rarity_strategy_version,rarity_rank,rarity_score,rarity_calculated_at,rarity_max_rank,rarity_tokens_scored,rarity_ranking_features
0,9394,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #9394,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmNzQ4uE2pmFpc8X1vSN5qouJ...,https://i.seadn.io/s/raw/files/7575f280a04f207...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,9445,None,,None,0,None
1,8722,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #8722,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmSoAHULJHoVTHiRyEEh53kEP...,https://i.seadn.io/s/raw/files/7a2a3413d03002f...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,9525,None,,None,0,None
2,9522,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #9522,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmatHsaQx6wxHry1vrETEMFvo...,https://i.seadn.io/s/raw/files/bdca83555861bef...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,5796,None,,None,0,None
3,63,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #63,A community-driven collectibles project featur...,https://ipfs.io/ipfs/Qmd1RdAChqxDrLRcgoYeMtiS1...,https://i.seadn.io/s/raw/files/5d12e77de0c9931...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,7564,None,,None,0,None
4,4941,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #4941,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmRZySdW7CgHznnkn4mHTeXxG...,https://i.seadn.io/s/raw/files/e859e07661ddd8d...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,8844,None,,None,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,4037,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #4037,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmYFY3St81wKfZobEMxpzdCPa...,https://i.seadn.io/s/raw/files/bf16a4de53cac3a...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,6737,None,,None,0,None
343,8618,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #8618,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmRD2HbD9MLczY7qMkDN4m8Zc...,https://i.seadn.io/s/raw/files/7a56ab76528d2e5...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,9082,None,,None,0,None
344,8547,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #8547,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmbnHaRW6Po61WLhc2EsVYr2P...,https://i.seadn.io/s/raw/files/6fdb11f09805727...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,1021,None,,None,0,None
345,4179,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #4179,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmYUazrP4x1cwwSGroqcz3vJr...,https://i.seadn.io/s/raw/files/9a49c6197097cf1...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,ethereum,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,None,None,2265,None,,None,0,None


In [ ]:
nfts_df = nfts_df.rename(columns=lambda x: f"nft_{x}")
# 保留你想保留的列，删除其余重复的列
nfts_df = nfts_df.drop(columns=['nft_nft_identifier', 'nft_nft_contract', 'nft_chain'])

,nft_identifier,nft_collection,nft_contract,nft_token_standard,nft_name,nft_description,nft_image_url,nft_display_image_url,nft_display_animation_url,nft_metadata_url,...,nft_traits,nft_owners,nft_rarity_strategy_id,nft_rarity_strategy_version,nft_rarity_rank,nft_rarity_score,nft_rarity_calculated_at,nft_rarity_max_rank,nft_rarity_tokens_scored,nft_rarity_ranking_features
0,9394,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #9394,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmNzQ4uE2pmFpc8X1vSN5qouJ...,https://i.seadn.io/s/raw/files/7575f280a04f207...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,"[{'trait_type': 'face', 'display_type': None, ...",[{'address': '0x620b70123fb810f6c653da7644b5dd...,None,None,9445,None,,None,0,None
1,8722,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #8722,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmSoAHULJHoVTHiRyEEh53kEP...,https://i.seadn.io/s/raw/files/7a2a3413d03002f...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x29469395eaf6f95920e59f858042f0...,None,None,9525,None,,None,0,None
2,9522,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #9522,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmatHsaQx6wxHry1vrETEMFvo...,https://i.seadn.io/s/raw/files/bdca83555861bef...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,"[{'trait_type': 'hair', 'display_type': None, ...",[{'address': '0x09010409154d4bf3ad22d0cbf5c9b9...,None,None,5796,None,,None,0,None
3,63,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #63,A community-driven collectibles project featur...,https://ipfs.io/ipfs/Qmd1RdAChqxDrLRcgoYeMtiS1...,https://i.seadn.io/s/raw/files/5d12e77de0c9931...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,"[{'trait_type': 'face', 'display_type': None, ...",[{'address': '0x29469395eaf6f95920e59f858042f0...,None,None,7564,None,,None,0,None
4,4941,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #4941,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmRZySdW7CgHznnkn4mHTeXxG...,https://i.seadn.io/s/raw/files/e859e07661ddd8d...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0xa6b37d0aeecc697be636b8b6f6ce73...,None,None,8844,None,,None,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,4037,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #4037,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmYFY3St81wKfZobEMxpzdCPa...,https://i.seadn.io/s/raw/files/bf16a4de53cac3a...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x6799eb21fd024c3546c8a2e17650df...,None,None,6737,None,,None,0,None
343,8618,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #8618,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmRD2HbD9MLczY7qMkDN4m8Zc...,https://i.seadn.io/s/raw/files/7a56ab76528d2e5...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,"[{'trait_type': 'face', 'display_type': None, ...",[{'address': '0x29469395eaf6f95920e59f858042f0...,None,None,9082,None,,None,0,None
344,8547,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #8547,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmbnHaRW6Po61WLhc2EsVYr2P...,https://i.seadn.io/s/raw/files/6fdb11f09805727...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x620b70123fb810f6c653da7644b5dd...,None,None,1021,None,,None,0,None
345,4179,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #417

In [22]:
# 2. 从 nfts_df 中挑出不重复的字段（排除 all_events_df 已经有的）
columns_to_use = [col for col in nfts_df.columns if col not in all_events_df.columns or col == "nft_identifier"]

# 3. 准备合并用的副本
nfts_subset = nfts_df[columns_to_use]

# 4. 合并
merged_df = all_events_df.merge(nfts_subset, how="left", on="nft_identifier")

In [23]:
merged_df

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft_traits,nft_owners,nft_rarity_strategy_id,nft_rarity_strategy_version,nft_rarity_rank,nft_rarity_score,nft_rarity_calculated_at,nft_rarity_max_rank,nft_rarity_tokens_scored,nft_rarity_ranking_features
0,sale,0xcb96928a158a8692fd30c6066f097c904ca7de7b52dc...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1743461579,1,0x2d198f5f809691a66bc487ca572b22937b231b07,0x05a64fc3398b49b1dcbb45109ff6debf2ea3dc24,0xdd05070239444b8d678e1a40067678a572768ba286de...,1743461579,...,"[{'trait_type': 'face', 'display_type': None, ...",[{'address': '0x620b70123fb810f6c653da7644b5dd...,None,None,9445,None,,None,0,None
1,sale,,ethereum,,1743454463,1,0x3772fad0e4de82f27620c946e7f52946332ea02c,0x85d2c733e11e095b0adaa3c055b31fdd4168af2c,0xc1b2cc2cbfa62b66276a1084f257eb49c7100f35a8f1...,1743454463,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x29469395eaf6f95920e59f858042f0...,None,None,9525,None,,None,0,None
2,sale,0xc9b32c5cda29594e2e8e4bdab548913ddb2e68e44910...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1743454451,1,0xa82801ddf9f5b974085e0450bebdd95365f5c3ab,0x3772fad0e4de82f27620c946e7f52946332ea02c,0x9879b2761b49f801f8d7346d45350ab3cb1b62e98bab...,1743454451,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x29469395eaf6f95920e59f858042f0...,None,None,9525,None,,None,0,None
3,sale,,ethereum,,1743454343,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xa82801ddf9f5b974085e0450bebdd95365f5c3ab,0x6841edf41d82618278d912b54851664388cd8f72f072...,1743454343,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x29469395eaf6f95920e59f858042f0...,None,None,9525,None,,None,0,None
4,sale,,ethereum,,1743454331,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x1ea27bce786a81022dfc156059771e8d3279a9a6,0x9f3a3cab23612a8ea31e12dbc452da9ab8a6c6101431...,1743454331,...,"[{'trait_type': 'hair', 'display_type': None, ...",[{'address': '0x09010409154d4bf3ad22d0cbf5c9b9...,None,None,5796,None,,None,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,sale,0x2e5cefee957f16fa19cca934ff30633a59a1f591ccbd...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1740838139,1,0xdddd696973fd58db7aec22f96fbdcf40371e9fb1,0x4a1721c493c481b48ab05db82265e149c95ac33f,0xbcbb4729168a2245b7a191cf84f1b33bb1fe8a584486...,1740838139,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x40d142a959a97b275eb03297a2e51c...,None,None,6796,None,,None,0,None
805,sale,,ethereum,,1740835607,1,0x0d53cb5115db5965b66a43f09692750de174fe91,0x6eced2947c674c388b638564848aadf2f05ea974,0x26f536f33d5df202e01159dedbe527384b170fa96118...,1740835607,...,"[{'trait_type': 'body', 'display_type': None, ...",[{'address': '0x27ec37f465e1755036cc854aa96dd3...,None,None,9777,None,,None,0,None
806,sale,0x7f8e16bcc01cbab03e7a35be421b5f2e7313dcf24960...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1740823991,1,0xf9bce92da0d4b8253077a6851a32bbd059e2a5a4,0x29e3c6eda0caf630b76493e3e6ea0adf9257f10c,0xf7612dc23a4bc7d6045ff7d4b7ec56a3d6d665d83a3b...,1740823991,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x29e3c6eda0caf630b76493e3e6ea0a...,None,None,3324,None,,None,0,None
807,sale,,ethereum,,1740821303,1,0x5b468edb7688e9ae6c1fa5a6d2debbef06e92907,0xf9091ba435a41f0d461d896cfea6f5e78ffb475e,0x7e31fddfe412929777c121366c9d8383e9609e41f8ed...,1740821303,...,"[{'trait_type': 'head', 'display_type': None, ...",[{'address': '0x29469395eaf6f95920e59f858042f0...,None,None,9525,None,,None,0,None


### Get Events(by collection) 并行化处理

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import time
import json
from queue import Queue
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

requests.packages.urllib3.disable_warnings()

# API 配置
API_KEY = "af06c28ee5414d31b2be1ab513d826af"  # 替换为你的 API 密钥
COLLECTION_SLUGS = [
    "boredapeyachtclub",
    "cryptopunks",
    "doodles-official",
]     ### 多个NFT

# 时间范围
dt1 = datetime(2025, 3, 1, 0, 0, 0, tzinfo=timezone.utc)
dt2 = datetime(2025, 4, 1, 0, 0, 0, tzinfo=timezone.utc)
AFTER_TIMESTAMP = int(dt1.timestamp())
BEFORE_TIMESTAMP = int(dt2.timestamp())

# 请求会话，带重试机制
session = requests.Session()
retries = Retry(total=3, backoff_factor=2, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

# 请求队列，控制速率
request_queue = Queue()

def fetch_collection_events(collection_slug):
    """获取单个 Collection 的交易事件"""
    url = f"https://api.opensea.io/api/v2/events/collection/{collection_slug}"
    headers = {
        "accept": "application/json",
        "x-api-key": API_KEY
    }
    params = {
        "after": AFTER_TIMESTAMP,
        "before": BEFORE_TIMESTAMP,
        "event_type": ["sale"],
        "limit": 50
    }
    
    all_events = []
    next_cursor = None
    
    while True:
        if next_cursor:
            params["next"] = next_cursor
            
        try:
            # 从队列获取许可，控制请求速率
            request_queue.get()
            response = session.get(url, headers=headers, params=params, verify=False, timeout=10)
            request_queue.put(1)  # 归还许可
            
            if response.status_code == 200:
                data = response.json()
                events = data.get("asset_events", [])
                for event in events:
                    event["collection_slug"] = collection_slug  # 添加标识
                all_events.extend(events)
                
                next_cursor = data.get("next")
                if not next_cursor:
                    break
            elif response.status_code == 429:
                print(f"Rate limit for {collection_slug}, waiting 10 seconds...")
                time.sleep(10)
                continue
            else:
                print(f"Error for {collection_slug}: {response.status_code}, {response.text}")
                break
                
        except requests.exceptions.RequestException as e:
            print(f"Request error for {collection_slug}: {e}")
            break
        finally:
            time.sleep(0.5)  # 每请求间隔 0.5 秒
            
    return collection_slug, all_events

# 并行获取
max_workers = min(3, len(COLLECTION_SLUGS))  # 动态设置最大线程数
results = {}

# 初始化请求队列，限制并发请求
for _ in range(max_workers):
    request_queue.put(1)

try:
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_slug = {
            executor.submit(fetch_collection_events, slug): slug
            for slug in COLLECTION_SLUGS
        }
        
        for future in concurrent.futures.as_completed(future_to_slug):
            slug = future_to_slug[future]
            try:
                collection_slug, events = future.result()
                if events:
                    df = pd.json_normalize(events, sep="_")
                    results[collection_slug] = df
                    print(f"Success: {collection_slug}, {len(df)} events")
                    # 即时保存
                    df.to_csv(f"events_{collection_slug}.csv", index=False)
                else:
                    print(f"No events for {collection_slug}")
            except Exception as e:
                print(f"Exception for {slug}: {e}")
                
except KeyboardInterrupt:
    print("Interrupted, saving results...")
    for slug, df in results.items():
        df.to_csv(f"events_{slug}.csv", index=False)

# 合并所有 DataFrame
if results:
    all_dfs = [df for df in results.values()]
    combined_events_df = pd.concat(all_dfs, ignore_index=True)

session.close()

Success: cryptopunks, 178 events
Success: boredapeyachtclub, 315 events
Success: doodles-official, 809 events

Collection: cryptopunks, 178 events

Collection: boredapeyachtclub, 315 events

Collection: doodles-official, 809 events


In [ ]:
combined_events_df
combined_events_df.to_csv("combined_events.csv", index=False)

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft_display_animation_url,nft_metadata_url,nft_opensea_url,nft_updated_at,nft_is_disabled,nft_is_nsfw,payment_quantity,payment_token_address,payment_decimals,payment_symbol
0,sale,,ethereum,,1743460691,1,0x4074bc05a89f1b97b51413b06f7e44f46eae6880,0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,0x539bc569dd63129e116146bc6e4740ebaa1fe47a0469...,1743460691,...,None,None,https://opensea.io/assets/ethereum/0xb47e3cd83...,2025-04-12T03:18:37.776967,False,False,45000000000000000000,0x0000000000000000000000000000000000000000,18,ETH
1,sale,,ethereum,,1743460619,1,0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,0xd59d37d76aaee5c70c76e748c7ade8de0dea216d,0x3422430c541fead1a3354b31df5720e38213252116cc...,1743460619,...,None,None,https://opensea.io/assets/ethereum/0xb47e3cd83...,2025-04-07T23:59:25.998849,False,False,42990000000000000000,0x0000000000000000000000000000000000000000,18,ETH
2,sale,,ethereum,,1743450311,1,0xcc2a855946a3c20683858fe6ee15acf8b836f0b3,0xec6ca6df559cb1617a29bc585ec6435ed35a32ed,0x23c8144550699fc40fff1ba3b2f32debbbce3a3639c7...,1743450311,...,None,None,https://opensea.io/assets/ethereum/0xb47e3cd83...,2025-04-05T18:25:50.364722,False,False,47990000000000000000,0x0000000000000000000000000000000000000000,18,ETH
3,sale,,ethereum,,1743449687,1,0x2238c8b16c36628b8f1f36486675c1e2a30debf1,0xec6ca6df559cb1617a29bc585ec6435ed35a32ed,0xd86ea49680258856aae3c291800a1cdb2a269307f9da...,1743449687,...,None,None,https://opensea.io/assets/ethereum/0xb47e3cd83...,2025-04-05T18:28:21.790247,False,False,46990000000000000000,0x0000000000000000000000000000000000000000,18,ETH
4,sale,,ethereum,,1743449279,1,0x14a191ce45818063cb5a06561538f95c34de5402,0xec6ca6df559cb1617a29bc585ec6435ed35a32ed,0xc85617b9bfbfaa80b278b8d0f67ac020d10d0a6e748a...,1743449279,...,None,None,https://opensea.io/assets/ethereum/0xb47e3cd83...,2025-04-05T18:27:41.661801,False,False,44990000000000000000,0x0000000000000000000000000000000000000000,18,ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,sale,0x2e5cefee957f16fa19cca934ff30633a59a1f591ccbd...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1740838139,1,0xdddd696973fd58db7aec22f96fbdcf40371e9fb1,0x4a1721c493c481b48ab05db82265e149c95ac33f,0xbcbb4729168a2245b7a191cf84f1b33bb1fe8a584486...,1740838139,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-03-04T16:34:49.723179,False,False,3200000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1298,sale,,ethereum,,1740835607,1,0x0d53cb5115db5965b66a43f09692750de174fe91,0x6eced2947c674c388b638564848aadf2f05ea974,0x26f536f33d5df202e01159dedbe527384b170fa96118...,1740835607,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-03-16T10:58:27.026730,False,False,3287900000000000000,0x0000000000000000000000000000000000000000,18,ETH
1299,sale,0x7f8e16bcc01cbab03e7a35be421b5f2e7313dcf24960...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1740823991,1,0xf9bce92da0d4b8253077a6851a32bbd059e2a5a4,0x29e3c6eda0caf630b76493e3e6ea0adf9257f10c,0xf7612dc23a4bc7d6045ff7d4b7ec56a3d6d665d83a3b...,1740823991,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-03-01T10:13:13.862119,False,False,3398000000000000000,0x0000000000000000000000000000000000000000,18,ETH
1300,sale,,ethereum,,1740821303,1,0x5b468edb7688e9ae6c1fa5a6d2debbef06e92907,0xf9091ba435a41f0d461d896cfea6f5e78ffb475e,0x7e31fddfe412929777c121366c9d8383e9609e41f8ed...,1740821303,...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,https://opensea.io/assets/ethereum/0x8a90cab2b...,2025-04-07T02:52:03.601317,False,False,3288498000000000000,0x0000000000000000000000000000000000000000,18,ETH


### 并行取多个collection的nft信息

In [ ]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import time
from queue import Queue
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from tqdm import tqdm
requests.packages.urllib3.disable_warnings()

# API 配置
API_KEY = "af06c28ee5414d31b2be1ab513d826af"  # 替换为你的 API 密钥

# 请求会话，带重试机制
session = requests.Session()
retries = Retry(total=3, backoff_factor=2, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

# 请求头
headers = {
    "accept": "application/json",
    "x-api-key": API_KEY
}

# 请求队列，控制速率
request_queue = Queue()

def fetch_nft(chain, contract, identifier):
    """获取单个 NFT 的信息"""
    url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{contract}/nfts/{identifier}"
    try:
        # 从队列获取许可
        request_queue.get()
        response = session.get(url, headers=headers, timeout=10, verify=False)
        request_queue.put(1)  # 归还许可
        
        if response.status_code == 200:
            data = response.json()
            if "nft" in data:
                nft = data["nft"]
                nft["nft_identifier"] = identifier
                nft["chain"] = chain
                nft["nft_contract"] = contract
                return nft
            return None
        elif response.status_code == 429:
            print(f"Rate limit for {identifier}, waiting 10 seconds...")
            time.sleep(10)
            return None
        else:
            print(f"Error for {identifier}: {response.status_code}, {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error for {identifier}: {e}")
        return None
    finally:
        time.sleep(0.5)  # 每请求间隔 0.5 秒

# 去重并准备数据
unique_nfts_df = combined_events_df[["nft_identifier", "chain", "nft_contract"]].drop_duplicates(subset="nft_identifier")
print(f"🔍 Total unique NFTs to fetch: {len(unique_nfts_df)}")

# 并行获取 NFT 信息
max_workers = 3  # 控制并发，OpenSea 免费密钥建议 2-3
results = []
fails = []
fetched_identifiers = set()

# 初始化请求队列
for _ in range(max_workers):
    request_queue.put(1)

try:
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 提交任务
        future_to_nft = {
            executor.submit(fetch_nft, row["chain"], row["nft_contract"], row["nft_identifier"]): row
            for _, row in unique_nfts_df.iterrows()
            if row["nft_identifier"] not in fetched_identifiers
        }
        
        # 收集结果
        for future in tqdm(concurrent.futures.as_completed(future_to_nft), total=len(future_to_nft), desc="Fetching NFTs"):
            row = future_to_nft[future]
            identifier = row["nft_identifier"]
            
            try:
                nft = future.result()
                if nft:
                    results.append(nft)
                    fetched_identifiers.add(identifier)
                    # 即时保存（每 1000 条）
                    if len(results) % 1000 == 0:
                        temp_df = pd.json_normalize(results, sep="_")
                        temp_df.to_csv("nfts_temp.csv", index=False)
                        print(f"Saved {len(results)} NFTs to nfts_temp.csv")
                else:
                    fails.append({"chain": row["chain"], "contract": row["nft_contract"], "identifier": identifier})
            except Exception as e:
                print(f"Exception for {identifier}: {e}")
                fails.append({"chain": row["chain"], "contract": row["nft_contract"], "identifier": identifier})

except KeyboardInterrupt:
    print("Interrupted, saving results...")
    if results:
        temp_df = pd.json_normalize(results, sep="_")
        temp_df.to_csv("nfts_temp.csv", index=False)
        print(f"Saved {len(results)} NFTs to nfts_temp.csv")
    if fails:
        pd.DataFrame(fails).to_csv("nfts_fails.csv", index=False)
        print(f"Saved {len(fails)} fails to nfts_fails.csv")

# 保存最终结果
if results:
    combined_nfts_df = pd.json_normalize(results, sep="_")
    combined_nfts_df.to_csv("nfts_final.csv", index=False)
    print(f"\nSaved {len(nfts_df)} NFTs to nfts_final.csv")
else:
    print("\nNo NFTs fetched")

if fails:
    fails_df = pd.DataFrame(fails)
    fails_df.to_csv("nfts_fails.csv", index=False)
    print(f"Saved {len(fails_df)} fails to nfts_fails.csv")

# 打印概要
print(f"\nFetched {len(results)} NFTs, {len(fails)} failed")

session.close()

🔍 Total unique NFTs to fetch: 651


Fetching NFTs: 100%|██████████| 651/651 [04:52<00:00,  2.23it/s]


Saved 651 NFTs to nfts_final.csv

Fetched 651 NFTs, 0 failed


In [36]:
combined_nfts_df = combined_nfts_df.rename(columns=lambda x: f"nft_{x}")
# 保留你想保留的列，删除其余重复的列
combined_nfts_df = combined_nfts_df.drop(columns=['nft_nft_identifier', 'nft_nft_contract', 'nft_chain'])

In [37]:
combined_nfts_df

,nft_identifier,nft_collection,nft_contract,nft_token_standard,nft_name,nft_description,nft_image_url,nft_display_image_url,nft_display_animation_url,nft_metadata_url,...,nft_owners,nft_rarity,nft_rarity_strategy_id,nft_rarity_strategy_version,nft_rarity_rank,nft_rarity_score,nft_rarity_calculated_at,nft_rarity_max_rank,nft_rarity_tokens_scored,nft_rarity_ranking_features
0,5500,cryptopunks,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,cryptopunks,CryptoPunk #5500,None,https://www.larvalabs.com/cryptopunks/cryptopu...,https://i.seadn.io/s/raw/files/aa193abab0fe897...,None,None,...,[{'address': '0xec6ca6df559cb1617a29bc585ec643...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5295,cryptopunks,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,cryptopunks,CryptoPunk #5295,None,https://www.larvalabs.com/cryptopunks/cryptopu...,https://i.seadn.io/gae/pFqMrIQejZGaHUO28JDLd7l...,None,None,...,[{'address': '0x0232d1083e970f0c78f56202b9a666...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6480,cryptopunks,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,cryptopunks,CryptoPunk #6480,None,https://www.larvalabs.com/cryptopunks/cryptopu...,https://i.seadn.io/s/raw/files/90b1bb38ea4e495...,None,None,...,[{'address': '0xd59d37d76aaee5c70c76e748c7ade8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5036,cryptopunks,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,cryptopunks,CryptoPunk #5036,None,https://www.larvalabs.com/cryptopunks/cryptopu...,https://i.seadn.io/gae/vJrhMpro_LaOx5i0v1EdBp1...,None,None,...,[{'address': '0xec6ca6df559cb1617a29bc585ec643...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2158,cryptopunks,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,cryptopunks,CryptoPunk #2158,None,https://www.larvalabs.com/cryptopunks/cryptopu...,https://i.seadn.io/gae/XpScpf1yhl_5T_40zy8s87a...,None,None,...,[{'address': '0xec6ca6df559cb1617a29bc585ec643...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,8618,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #8618,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmRD2HbD9MLczY7qMkDN4m8Zc...,https://i.seadn.io/s/raw/files/7a56ab76528d2e5...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,[{'address': '0x29469395eaf6f95920e59f858042f0...,NaN,NaN,NaN,9082.0,NaN,,NaN,0.0,NaN
647,8547,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #8547,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmbnHaRW6Po61WLhc2EsVYr2P...,https://i.seadn.io/s/raw/files/6fdb11f09805727...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,[{'address': '0x620b70123fb810f6c653da7644b5dd...,NaN,NaN,NaN,1021.0,NaN,,NaN,0.0,NaN
648,4179,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #4179,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmYUazrP4x1cwwSGroqcz3vJr...,https://i.seadn.io/s/raw/files/9a49c6197097cf1...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,[{'address': '0x620b70123fb810f6c653da7644b5dd...,NaN,NaN,NaN,2265.0,NaN,,NaN,0.0,NaN
649,2151,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #2151,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmV6nzeZ5cMmJ968WS37LmyEE...,https://i.seadn.io/s/raw/files/f9caf50a5683e6f...,None,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,[{'address': '0x29e3c6eda0caf630b76493e3e6ea0a...,NaN,NaN,NaN,3324.0,NaN,,NaN,0.0,NaN


In [ ]:
# 2. 从 nfts_df 中挑出不重复的字段（排除 all_events_df 已经有的）
columns_to_use = [col for col in combined_nfts_df.columns if col not in combined_events_df.columns or col == "nft_identifier"]

# 3. 准备合并用的副本
nfts_subset = nfts_df[columns_to_use]

# 4. 合并
final_merged_df = combined_events_df.merge(nfts_subset, how="left", on="nft_identifier")

In [ ]:
final_merged_df.to_csv("final_merged_df.csv", index=False)
final_merged_df

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft_owners,nft_rarity,nft_rarity_strategy_id,nft_rarity_strategy_version,nft_rarity_rank,nft_rarity_score,nft_rarity_calculated_at,nft_rarity_max_rank,nft_rarity_tokens_scored,nft_rarity_ranking_features
0,sale,,ethereum,,1743460691,1,0x4074bc05a89f1b97b51413b06f7e44f46eae6880,0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,0x539bc569dd63129e116146bc6e4740ebaa1fe47a0469...,1743460691,...,[{'address': '0x0232d1083e970f0c78f56202b9a666...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sale,,ethereum,,1743460619,1,0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,0xd59d37d76aaee5c70c76e748c7ade8de0dea216d,0x3422430c541fead1a3354b31df5720e38213252116cc...,1743460619,...,[{'address': '0xd59d37d76aaee5c70c76e748c7ade8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sale,,ethereum,,1743450311,1,0xcc2a855946a3c20683858fe6ee15acf8b836f0b3,0xec6ca6df559cb1617a29bc585ec6435ed35a32ed,0x23c8144550699fc40fff1ba3b2f32debbbce3a3639c7...,1743450311,...,[{'address': '0xec6ca6df559cb1617a29bc585ec643...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sale,,ethereum,,1743449687,1,0x2238c8b16c36628b8f1f36486675c1e2a30debf1,0xec6ca6df559cb1617a29bc585ec6435ed35a32ed,0xd86ea49680258856aae3c291800a1cdb2a269307f9da...,1743449687,...,[{'address': '0xec6ca6df559cb1617a29bc585ec643...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sale,,ethereum,,1743449279,1,0x14a191ce45818063cb5a06561538f95c34de5402,0xec6ca6df559cb1617a29bc585ec6435ed35a32ed,0xc85617b9bfbfaa80b278b8d0f67ac020d10d0a6e748a...,1743449279,...,[{'address': '0xec6ca6df559cb1617a29bc585ec643...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,sale,0x2e5cefee957f16fa19cca934ff30633a59a1f591ccbd...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1740838139,1,0xdddd696973fd58db7aec22f96fbdcf40371e9fb1,0x4a1721c493c481b48ab05db82265e149c95ac33f,0xbcbb4729168a2245b7a191cf84f1b33bb1fe8a584486...,1740838139,...,[{'address': '0x40d142a959a97b275eb03297a2e51c...,NaN,NaN,NaN,6796.0,NaN,,NaN,0.0,NaN
1298,sale,,ethereum,,1740835607,1,0x0d53cb5115db5965b66a43f09692750de174fe91,0x6eced2947c674c388b638564848aadf2f05ea974,0x26f536f33d5df202e01159dedbe527384b170fa96118...,1740835607,...,[{'address': '0x27ec37f465e1755036cc854aa96dd3...,NaN,NaN,NaN,9777.0,NaN,,NaN,0.0,NaN
1299,sale,0x7f8e16bcc01cbab03e7a35be421b5f2e7313dcf24960...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1740823991,1,0xf9bce92da0d4b8253077a6851a32bbd059e2a5a4,0x29e3c6eda0caf630b76493e3e6ea0adf9257f10c,0xf7612dc23a4bc7d6045ff7d4b7ec56a3d6d665d83a3b...,1740823991,...,[{'address': '0x29e3c6eda0caf630b76493e3e6ea0a...,NaN,NaN,NaN,3324.0,NaN,,NaN,0.0,NaN
1300,sale,,ethereum,,1740821303,1,0x5b468edb7688e9ae6c1fa5a6d2debbef06e92907,0xf9091ba435a41f0d461d896cfea6f5e78ffb475e,0x7e31fddfe412929777c121366c9d8383e9609e41f8ed...,1740821303,...,[{'address': '0x20b5ec6f5d6faf79f98eb1592d13f0...,NaN,NaN,NaN,9525.0,NaN,,NaN,0.0,NaN
